<a href="https://colab.research.google.com/github/alicia0928/Resnet50/blob/main/%EB%AA%A8%EB%8D%B8_%EB%B6%88%EB%9F%AC%EC%98%A4%EB%8A%94_%ED%95%A8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 transform
from torch.utils.data import DataLoader # 이미지 데이터 로더
import os
import glob

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 경로 지정

train_path = '/content/drive/MyDrive/파이토치/train'
test_path = '/content/drive/MyDrive/파이토치/test'

Dataset_path = '/content/drive/MyDrive/파이토치'

cactus_dir = '/cactus/'
dracaena_sanderiana_dir = '/dracaena_sanderiana/'
monstera_dir = '/monstera/'
rosemary_dir = '/rosemary/'

In [ ]:
# import resnet

from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim

resnet50 = models.resnet50(pretrained=False).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
resnet50.fc = nn.Linear(resnet50.fc.in_features, 4).to(device)

In [ ]:
from torch.utils.data import Dataset, DataLoader # 데이터 커스터마이징
from PIL import Image # PIL = Python Image Library
import cv2 # albumentation transform을 쓰려면 꼭 이 라이브러리를 이용
import tensorflow as tf

class inhovation_Dataset(Dataset):

  def __init__(self, file_path, mode, transform=None):
    self.all_data = sorted(glob.glob(os.path.join(file_path, mode, '*', '*')))
    self.transform = transform

  def __getitem__(self, index):

    if torch.is_tensor(index):        # 인덱스가 tensor 형태일 수 있으니 리스트 형태로 바꿔준다.
       index = index.tolist()

    data_path = self.all_data[index]
    img = np.array(Image.open(data_path).convert("RGB")) # albumenatation transform을 쓰려면 cv2 라이브러리로 이미지를 읽어야 함
    image=cv2.imread(data_path)
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환

    # transform 적용
    if self.transform is not None:
       augmented = self.transform(image=image)
       image = augmented['image']

    # 이미지 이름을 활용해 label 부여
    label=[]
    if os.path.basename(data_path).startswith("cactus") == True:
        label = 0
    elif os.path.basename(data_path).startswith("dracaena_sanderiana") == True:
        label = 1
    elif os.path.basename(data_path).startswith("monstera") == True:
        label = 2
    elif os.path.basename(data_path).startswith("rosemary") == True:
        label = 3
    else :
        label = 4
    return image, label

  def __len__(self):
    length = len(self.all_data)
    return length


In [ ]:
# import resnet
import torchvision.models.resnet as resnet
import torch.nn as nn
import torch.optim as optim

# 미리 정의
conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock

In [ ]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 32 # conv1에서 나올 채널의 차원 -> 이미지넷보다 작은 데이터이므로 32로 조정

        # inputs = 3x224x224 -> 3x128x128로 바뀜
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False) # 마찬가지로 전부 사이즈 조정
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 32, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2) # 3 반복

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import albumentations
from albumentations.pytorch.transforms import ToTensorV2


albumentations_resize = albumentations.Compose([

    albumentations.Resize(128, 128),
    ToTensorV2()

])

# resize_train=inhovation_Dataset(Dataset_path, 'train', transform=albumentations_resize)
# resize_test=inhovation_Dataset(Dataset_path, 'test', transform=albumentations_resize)
resize_test_mean=[0.15918699, 0.410329, 0.55247366]
resize_test_std=[0.1542138, 0.16098696, 0.15552239]
# # 이미지 전처리를 위한 변환 정의
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(resize_test_mean, resize_test_std)
])


# 이미지 불러오기
image = Image.open('/content/drive/MyDrive/파이토치/model_test/test6.jpg')

# 전처리 적용
input_tensor = transform_test(image)  # 전처리된 이미지 텐서

# 배치 차원 추가
input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가

# GPU 사용 가능 여부 확인 및 적용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_batch = input_batch.to(device)

# 모델 불러오기
loaded_model = torch.load('/content/drive/MyDrive/파이토치/best_model.pth', map_location=torch.device('cpu'))


# 모델을 평가 모드로 전환
loaded_model.eval()

# 모델에 입력 전달하여 출력 얻기
with torch.no_grad():  # 그래디언트 계산 비활성화
    output = loaded_model(input_batch)

# 출력값 중 가장 높은 값을 갖는 클래스 선택
_, predicted = torch.max(output, 1)

# 선택된 클래스 출력
print("예측된 클래스:", predicted.item())

예측된 클래스: 2
